<a href="https://colab.research.google.com/github/maharshi-roy/Machine-Learning-Experiments/blob/main/Neural_Machine_Translation_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Make imports
import numpy as np
import re
import pickle
import os
import seaborn as sns
import string

In [2]:
#TPU settings
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.6.0
Running on TPU  ['10.76.194.114:8470']
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.76.194.114:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.76.194.114:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [3]:
def preprocess(text):
  text = ''.join(ch for ch in text if ch not in string.punctuation)
  text = text.lower()
  text = re.sub(r'\d','',text)
  text = re.sub(r'\s+',' ',text)
  text = text.strip()
  return text

In [5]:
#Extract dataset and preprocess
dataset_root = "./drive/MyDrive/ml/datasets/"

if os.path.exists(dataset_root + "parallel/preprocessed_data.pickle"):
  with open(dataset_root + "parallel/preprocessed_data.pickle", 'rb') as f:
    english_sentences, hindi_sentences = pickle.load(f)
else:
  if not os.path.exists(dataset_root + "parallel/IITB.en-hi.en"):
    os.system("tar -xzf " + dataset_root + "parallel.tgz -C " + dataset_root)

  with open(dataset_root + "parallel/IITB.en-hi.en",'r') as f:
    english_sentences = f.read().split('\n')

  with open(dataset_root + "parallel/IITB.en-hi.hi",'r') as f:
    hindi_sentences = f.read().split('\n')

  english_sentences = [preprocess(en) for en in english_sentences]
  hindi_sentences = ['<START> ' + re.sub('[a-zA-Z]','',preprocess(hi)) + ' <END>' for hi in hindi_sentences]

  with open(dataset_root + "parallel/preprocessed_data.pickle",'wb') as f:
    pickle.dump((english_sentences, hindi_sentences), f)

In [25]:
#Some parameters
vocab_size = 10000
total_sentences = 100000
maxlen = 20
epochs = 10

In [26]:
en_data = []
hi_data = []

cnt = 0

for (en,hi) in zip(english_sentences, hindi_sentences):
  l = min(len(en.split()), len(hi.split()))
  if l <= maxlen:
    en_data.append(en)
    hi_data.append(hi)
    cnt += 1
  if cnt == total_sentences:
    break

In [27]:
#Tokenize the texts and convert to sequences
en_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
en_tokenizer.fit_on_texts(en_data)
en_sequences = en_tokenizer.texts_to_sequences(en_data)

hi_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', oov_token='<OOV>', lower=False)
hi_tokenizer.fit_on_texts(hi_data)
hi_sequences = hi_tokenizer.texts_to_sequences(hi_data)

english_vocab_size = len(en_tokenizer.word_index) + 1
hindi_vocab_size = len(hi_tokenizer.word_index) + 1
print("English Vocab Size: ", english_vocab_size)
print("Hindi Vocab Size: ", hindi_vocab_size)

English Vocab Size:  1967
Hindi Vocab Size:  2046


In [28]:
#Prepare encoder data
encoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(en_sequences, maxlen=maxlen, padding='post')

In [29]:
#Prepare decoder data
decoder_inputs = []
decoder_outputs = []

for hi in hi_sequences:
  decoder_inputs.append(hi[:-1])
  decoder_outputs.append(hi[1:])

decoder_inputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_inputs, maxlen=maxlen, padding='post')
decoder_outputs = tf.keras.preprocessing.sequence.pad_sequences(decoder_outputs, maxlen=maxlen, padding='post')

In [30]:
# Training and Testing split
# 95%, 5%
split = int(0.95 * total_sentences)

X_train = [encoder_inputs[:split], decoder_inputs[:split]]
y_train = decoder_outputs[:split]

# Test data to evaluate our NMT model using BLEU score
X_test = en_data[split:]
y_test = hi_data[split:]

print(X_train[0].shape, X_train[1].shape, y_train.shape)

(23750, 20) (23750, 20) (23750, 20)


In [31]:
#Define LSTM model
d_model = 256

#Encoder
inputs = tf.keras.layers.Input(shape=(None,))
x = tf.keras.layers.Embedding(english_vocab_size, d_model, mask_zero=True)(inputs)
x, state_h, state_c = tf.keras.layers.LSTM(d_model,activation='relu',return_state=True)(x)

#Decoder
targets = tf.keras.layers.Input(shape=(None,))
embedding_layer = tf.keras.layers.Embedding(hindi_vocab_size, d_model, mask_zero=True)
x = embedding_layer(targets)
decoder_lstm = tf.keras.layers.LSTM(d_model,activation='relu',return_sequences=True, return_state=True)
x,_,_ = decoder_lstm(x, initial_state=[state_h, state_c])
dense1 = tf.keras.layers.Dense(hindi_vocab_size, activation='softmax')
x = dense1(x)

model = tf.keras.models.Model(inputs=[inputs, targets],outputs=x)
model.summary()

loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(optimizer='rmsprop', loss=loss, metrics=['accuracy'])

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 256)    503552      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 256)    523776      input_8[0][0]                    
____________________________________________________________________________________________

In [32]:
#Create callback to cancel training
class TerminateTraining(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs['accuracy'] >= 0.95:
      self.model.stop_training = True

In [33]:
model.fit(X_train, y_train, epochs=epochs, validation_split=0.05, callbacks=[TerminateTraining()])
model.save('./drive/MyDrive/ml/models/en-hi.h5')

Epoch 1/100
  6/706 [..............................] - ETA: 52s - loss: 24.1227 - accuracy: 0.2086WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_train_batch_end` time: 0.1917s). Check your callbacks.


706/706 [==============================] - 55s 73ms/step - loss: 1.0968 - accuracy: 0.2983 - val_loss: 0.6632 - val_accuracy: 0.3621
Epoch 2/100
706/706 [==============================] - 51s 73ms/step - loss: 0.6317 - accuracy: 0.4534 - val_loss: 0.4546 - val_accuracy: 0.5325
Epoch 3/100
706/706 [==============================] - 51s 73ms/step - loss: 0.4644 - accuracy: 0.5780 - val_loss: 0.2838 - val_accuracy: 0.6889
Epoch 4/100
706/706 [==============================] - 52s 73ms/step - loss: 0.3253 - accuracy: 0.6963 - val_loss: 0.1627 - val_accuracy: 0.8134
Epoch 5/100
706/706 [==============================] - 51s 73ms/step - loss: 0.2237 - accuracy: 0.7882 - val_loss: 0.0995 - val_accuracy: 0.8835
Epoch 6/100
706/706 [==============================] - 51s 73ms/step - loss: 0.1537 - accuracy: 0.8506 - val_loss: 0.0526 - val_accuracy: 0.9306
Epoch 7/100
706/706 [==============================] - 52s 73ms/step - loss: 0.1083 - accuracy: 0.8957 - val_loss: 0.0293 - val_accuracy: 0.95

In [34]:
#Inference Model

#Encoder
encoder = tf.keras.models.Model(inputs, [state_h, state_c])

#Decoder
decoder_input_h = tf.keras.layers.Input(shape=(d_model,))
decoder_input_c = tf.keras.layers.Input(shape=(d_model,))
x = embedding_layer(targets)
x, decoder_output_h, decoder_output_c = decoder_lstm(x, initial_state=[decoder_input_h, decoder_input_c])
x = dense1(x)
decoder = tf.keras.models.Model([targets] + [decoder_input_h, decoder_input_c], 
                                [x] + [decoder_output_h, decoder_output_c])

In [35]:
def predict_sentence(en_input):
  input_seq = en_tokenizer.texts_to_sequences([en_input])

  next_h, next_c = encoder.predict(input_seq)

  curr_token = np.zeros(1)
  curr_token[0] = hi_tokenizer.word_index['<START>']

  pred_sentence = ''

  for i in range(maxlen):
    output, next_h, next_c = decoder.predict([curr_token] + [next_h, next_c])
    next_token = np.argmax(output[0, 0, :])
    next_word = hi_tokenizer.index_word[next_token]
    if next_word == '<END>':
      break
    else:
      pred_sentence += ' ' + next_word
      curr_token[0] = next_token

  return pred_sentence

In [36]:
#Set up google translate as an additional reference
from google.cloud import translate_v2 as translate

translate_client = translate.Client.from_service_account_json("./drive/MyDrive/gcloud-auth-files/data-shard-330609-2669b6f2c900.json")

In [37]:
#Testing and Analysis
import nltk

candidates = []
references = []

for i in range(20):
  pred_sentence = predict_sentence(X_test[i])

  candidates.append(pred_sentence.split())
  google_translated_sentence = translate_client.translate(X_test[i], target_language='hi')['translatedText']
  references.append([y_test[i].split()[1:-1], google_translated_sentence.split()])

print(nltk.translate.bleu_score.corpus_bleu(references, candidates))

Candidate:  ['फ़ाइल', 'चुनें']
References:  [['फ़ाइल', 'चुनें'], ['फ़ाइल', 'का', 'चयन', 'करें']]
0.5210492912666359
